In [ ]:
import tensorflow as tf

## Check your runtime environment (optional)

Usually it is recommended to use GPU to train a CNN. Based on our dataset, it takes ~1min to train a single epoch using CPU, while only 10s when using a GPU.

In [ ]:
tf.test.gpu_device_name()  # check available gpu

## Load Dataset

In [ ]:
from tensorflow.keras.datasets import mnist
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### Get familiar with your data

In [ ]:
y_train.shape

In [ ]:
y_train[:5]

In [ ]:
Counter(y_train)  # Check the distribution of classess

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
def show_image(image):
  """A helper to display grey scale image"""
  plt.imshow(image, cmap='Greys')

In [ ]:
show_image(x_train[6])

In [ ]:
x_train[0]

## Preprocessing

- Reshape dataset by adding channel dimension (as required by Keras API)
- Normalize data

In [ ]:
# Add channel dimension and asting the dtype to float for decimal computation

x_train = np.expand_dims(x_train, -1).astype('float32')  # Add one dim to the end
x_test = np.expand_dims(x_test, -1).astype('float32')

print('Shape of training data:', x_train.shape)
print('Data type of training data:', x_train.dtype)

In [ ]:
# Normalization
x_train /= 255.0
x_test /= 255.0

## Build the Convolutional Neural Network

We will build a netowrk with the following layers:
- Convolutional layer with 28 3X3 kernels
- Max Pooling layer with 2X2 tile
- Flatten layer to convert 2D image into a 1D vector
- Fully connected layer with 128 units
- Output layer with softmax

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
model = Sequential()

model.add(Conv2D(filters=28, kernel_size=(3, 3), input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=0.2))  # 0.5 probability to drop a neuron
model.add(Dense(10, activation='softmax'))  # 10 possible output classes

In [ ]:
model.summary()

## Compile and train the model

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_test, y_test),
    batch_size=64,
    epochs=10
)

In [ ]:
def plot_history(his, metrics):
  """
  Given a history object returned from `fit` and the name of metrics,
  plot the curve of metrics aginst number of epochs.
  """
  for metric in metrics:
    plt.plot(his.history[metric], label=metric)
  plt.legend()

In [ ]:
plot_history(history, ['loss', 'val_loss'])

In [ ]:
plot_history(history, ['accuracy', 'val_accuracy'])

## Predict

In [ ]:
def predict_image(model, image):
  # Apply the same pre-processing
  show_image(image)
  pred = model.predict(image.reshape(1, 28, 28, 1))  # reshape to (batch, img_rows, img_cols, channel)
  print("Raw prediction output:", pred)
  print("Predicted number:", pred.argmax())

In [ ]:
(_, _), (test_images, _) = mnist.load_data()

In [ ]:
from random import randint

test_image = test_images[randint(1, test_images.shape[0])]
predict_image(model, test_image)